## Классификация отзывов (домашнее задание)


### Загрузка необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier #модуль для построения линейных моделей
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn import metrics

### Загружаем данные

In [6]:
df = pd.read_csv("reviews.csv")

### Исследуем наш датасет

(заодно потренеруемся в работе с pandas)

In [7]:
#Посмотрим на объем датасета (сколько в нем строк и колонок): 
df.shape

(46501, 2)

In [8]:
#Посмотрим шапку:
df.head()

,label,text
0,1,Эпиграф Добро которое ты делаешь от сердца ты ...
1,1,Теперь это один из моих любимых фильмов в жанр...
2,1,Что скрыто в фильме Лучше не бывает Одна шикар...
3,1,Перед нами очень милое и доброе кино которое л...
4,1,Завязка Мелвин Удал популярный писатель Нет не...


In [10]:
#Посмотрим, какие значения есть в Labels и как количественно распределяются
df['label'].value_counts()

 1    36480
 0     5645
-1     4376
Name: label, dtype: int64

### Векторизация текстов

Векторизируем текст с помощью CountVectorizer c настройками по умолчанию

In [11]:
#Сначала инициализируем CountVectorizer:
vec = CountVectorizer()
#Сделаем матрицу из нашего текста (мешок слов BagOfWords), то есть берем текст из колонки text:
bow = vec.fit_transform(df.text)

In [12]:
#Посмотрим, как выглядит признаковое представление нашего текста:
print(bow)

  (0, 366655)	1
  (0, 70309)	1
  (0, 124109)	2
  (0, 329109)	4
  (0, 64758)	2
  (0, 197067)	7
  (0, 290666)	1
  (0, 41924)	1
  (0, 289185)	2
  (0, 323513)	1
  (0, 75776)	1
  (0, 163170)	18
  (0, 220963)	1
  (0, 259033)	1
  (0, 106598)	1
  (0, 78333)	9
  (0, 293257)	1
  (0, 357853)	11
  (0, 136044)	1
  (0, 367689)	2
  (0, 344796)	2
  (0, 367672)	5
  (0, 300485)	2
  (0, 195082)	1
  (0, 77977)	2
  :	:
  (46500, 43241)	1
  (46500, 263498)	1
  (46500, 91943)	1
  (46500, 93785)	1
  (46500, 311339)	1
  (46500, 16669)	1
  (46500, 161418)	1
  (46500, 297299)	1
  (46500, 230425)	1
  (46500, 346196)	1
  (46500, 310277)	1
  (46500, 350057)	1
  (46500, 82140)	1
  (46500, 222345)	1
  (46500, 268600)	1
  (46500, 318074)	1
  (46500, 258568)	1
  (46500, 230334)	1
  (46500, 244816)	1
  (46500, 304990)	1
  (46500, 227452)	1
  (46500, 361464)	1
  (46500, 55383)	1
  (46500, 218775)	1
  (46500, 9487)	1


In [13]:
bow

<46501x369844 sparse matrix of type '<class 'numpy.int64'>'
	with 11693675 stored elements in Compressed Sparse Row format>

In [15]:
#Проверим длину текстов:
len(df.text)

46501

In [14]:
#Посмотрим на количество признаков в матрице (просто вызов bow дает нам ту же информацию): 
print('Количество признаков-частот слов в CountVectorizer:' + str(bow.shape[1]))

Количество признаков-частот слов в CountVectorizer:369844


### Сделаем тестовые и обучающие выборки

Мы будем обучать модель на обучающей выборке и проверять ее качество на тесте.
Чтобы собрать тестовую и обучающую выборки из исходных данных, используем функцию кросс-валидации train_test_split, реализованной в scikit-learn. Она позволяет нам построить разовое разбиение данных на обучение и тест.

In [17]:
#В качестве аргумента функция принимает набор данных, которые мы хотим разбить (bow), 
#набор меток классов (df.label), и также ей можно указать соотношение, в котором мы хотим разбивать данные

train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 

In [18]:
#Смотрим на размер обучающей выборки
train_data

<34875x369844 sparse matrix of type '<class 'numpy.int64'>'
	with 8764370 stored elements in Compressed Sparse Row format>

In [19]:
#Смотрим на размер тестовой выборки: 
test_data

<11626x369844 sparse matrix of type '<class 'numpy.int64'>'
	with 2929305 stored elements in Compressed Sparse Row format>

## Логистическая регрессия (с настройками по умолчанию)

In [20]:
#Создание объекта-классификатора (модель). Строим логистическую регрессию и используем  для этого класс LogisticRegression:
lr = LogisticRegression(random_state=1)

In [21]:
#Обучение нашего классификатора (модели). Передаем данные, на которых нужно обучаться (наш векторизированный текст) и метки классов: 
lr.fit(train_data, train_labels)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Оценка качества модели

In [22]:
#Применение обученного классификатора. Cтроим предсказания с помощью метода predict:
predicted_labels = lr.predict(test_data)

In [23]:
#Посмотрим метки на тестовой выборке
print(test_labels)

31355    1
3028     1
24927    1
43461    1
23369    1
3952     1
4103     1
31972    0
10975    1
25047    1
22469    1
41507    1
46453    1
28542   -1
43194    1
2971     1
25347    0
32681   -1
30127    0
32074   -1
33435    0
4393     1
9602     1
17806   -1
10098    1
42048    1
33922    0
16498    1
10402    1
36737    1
        ..
8899     1
21041    1
15245    1
14991    1
21671    1
33505    1
1851     1
8621    -1
4980     1
42075    1
23205    1
33565    1
11464    1
11289    1
3409     1
34303   -1
32239   -1
29774   -1
3117     1
18311    1
32630   -1
20242    1
20675    1
4258     1
4827     1
15595    1
12981    1
30423    1
8604     1
42831    1
Name: label, Length: 11626, dtype: int64


In [24]:
#Сравним с предсказаниями, сделанными нашей моделью на тестовой выборке:
predicted_labels

array([1, 1, 1, ..., 1, 1, 1])

Когда мы используем логистическую регрессию, мы работаем с вероятностной моделью. 
Помимо меток классов, эта модель может выдать нам вероятности, с которыми каждый объект принадлежит классу 0, 1 или -1.

In [25]:
#Построим эти вероятности. Для этого нужно воспользоваться методом predict_proba (или predict probability). 
#В качестве аргумента передаем тестовую выборку и получаем наши вероятности принадлежности к классам.
lr.predict_proba(test_data) 

#Мы видим, что для каждого объекта нам доступны следующие значения: вероятность принадлежности объекта к классу 0, -1, 1

array([[1.91623084e-04, 1.11482831e-02, 9.88660094e-01],
       [4.98672807e-07, 4.71632279e-04, 9.99527869e-01],
       [1.23753931e-04, 1.13042469e-04, 9.99763204e-01],
       ...,
       [4.06295504e-04, 8.00816177e-05, 9.99513623e-01],
       [3.10519133e-03, 4.01203704e-04, 9.96493605e-01],
       [8.76832176e-03, 1.65094714e-02, 9.74722207e-01]])

In [26]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.65      0.50      0.57      1114
           0       0.39      0.25      0.30      1413
           1       0.88      0.95      0.91      9099

    accuracy                           0.82     11626
   macro avg       0.64      0.57      0.60     11626
weighted avg       0.80      0.82      0.81     11626



In [27]:
my_vec = vec.transform(['Не советую смотреть данный фильм: плохая игра актеров, примитивный сценарий!'])

In [28]:
lr.predict(my_vec) #Тут модель неверно предсказала. 

array([1])

In [29]:
#Посмотрим на 10 наиболее важных признаков для модели:
coeffs = lr.coef_[0]
feats10 = [vec.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[:10]]
feats10

['отличный',
 'приятно',
 'отлично',
 'самих',
 'лучших',
 'пересматривать',
 'дыхании',
 'становятся',
 'слегка',
 'спорю']

### Попробуем выставить параметр solver = 'newton-cg' в  логрегрессии: 

solver = 'newton-cg'применяют для мультиклассовой классификации

In [30]:
lr_newton = LogisticRegression(random_state=1, solver = 'newton-cg')

In [31]:
#Обучение нашего классификатора
lr_newton.fit(train_data, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
#Строим предсказания на тестовой выборке:
predicted_labels = lr.predict(test_data)

In [33]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.65      0.50      0.57      1114
           0       0.39      0.25      0.30      1413
           1       0.88      0.95      0.91      9099

    accuracy                           0.82     11626
   macro avg       0.64      0.57      0.60     11626
weighted avg       0.80      0.82      0.81     11626



Качество по accuracy не изменилось.

### Векторизируем текст, используя биграммы

In [34]:
#Инициализируем CountVectorizer, используя биграммы:
vec = CountVectorizer(ngram_range=(2, 2))
bow = vec.fit_transform(df.text)
train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 

In [35]:
#Обучаем модель с биграммами
lr = LogisticRegression(random_state=1)
lr.fit(train_data, train_labels)
predicted_labels = lr.predict(test_data)
print(classification_report(test_labels, predicted_labels))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.68      0.35      0.46      1070
           0       0.48      0.11      0.18      1425
           1       0.84      0.99      0.91      9131

    accuracy                           0.82     11626
   macro avg       0.66      0.48      0.51     11626
weighted avg       0.78      0.82      0.78     11626



Качество по accuracy не изменилось.

### Векторизуем текст, используя токенизатор из NLTK, а также список стоп-слов оттуда

In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [37]:
rus_stopwords = stopwords.words('russian')

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=rus_stopwords) #используем токенизатор из нлтк и стоп слова
bow = vec.fit_transform(df.text)
train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 
lr = LogisticRegression(random_state=1)
lr.fit(train_data, train_labels)
predicted_labels = lr.predict(test_data)
print(classification_report(test_labels, predicted_labels))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.66      0.51      0.58      1085
           0       0.36      0.22      0.27      1413
           1       0.88      0.95      0.91      9128

    accuracy                           0.82     11626
   macro avg       0.63      0.56      0.59     11626
weighted avg       0.79      0.82      0.80     11626



Качество по accuracy не изменилось.

### Использование TF-IDF векторизации

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(df.text)
train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 
lr = LogisticRegression(random_state=1)
lr.fit(train_data, train_labels)
predicted_labels = lr.predict(test_data)
print(classification_report(test_labels, predicted_labels))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.74      0.36      0.48      1081
           0       0.49      0.07      0.12      1314
           1       0.84      0.99      0.91      9231

    accuracy                           0.83     11626
   macro avg       0.69      0.47      0.50     11626
weighted avg       0.79      0.83      0.78     11626



#### Качество с TfidfVectorizer чуть лучше!

### Подбор параметров для логистической регрессии с помощью GridSearchCV 

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
vec = CountVectorizer()
bow = vec.fit_transform(df.text)
train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 
lr = LogisticRegression(random_state=1)

In [49]:
#Посмотрим, какие параметры мы можем подобрать: 
lr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

penalty - регуляризация
С - обратный коэффициент регуляризации

In [50]:
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_lr_acc = GridSearchCV(lr, param_grid = grid_values, scoring = 'accuracy')

In [51]:
grid_lr_acc.fit(train_data, train_labels)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=1, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.009, 0.01, 0.09, 1, 5, 10, 25],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [52]:
#Посмотрим на лучший классификатор (возвращает модель с лучшими параметрами):
grid_lr_acc.best_estimator_

LogisticRegression(C=0.09, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
#Отдельно можно посмотреть на оценку на лучшем наборе параметров:
print(grid_lr_acc.best_score_)

#Вывести лучшие наборы параметров: 
print(grid_lr_acc.best_params_)

0.8299928315412186
{'C': 0.09, 'penalty': 'l2'}


In [ ]:
#Чтобы посмотреть оценки на первых 10 наборах:  #Не работает!
grid_lr_acc.grid_scores_[:10]

Обучим логистическую регрессию снова, выставив параметры, рекомендованные GridSearch. Регуляризация L2 стоит по умолчанию.

In [55]:
lr = LogisticRegression(random_state=1,C = 0.09)
lr.fit(train_data, train_labels)
predicted_labels = lr.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.69      0.53      0.60      1081
           0       0.42      0.21      0.28      1382
           1       0.88      0.97      0.92      9163

    accuracy                           0.84     11626
   macro avg       0.66      0.57      0.60     11626
weighted avg       0.81      0.84      0.81     11626



#### Нам удалось поднять качество у логрегрессии на 0.2!

In [58]:
#Протестируем какой-нибудь отрывок из отзыва из Интернета:
my_vec = vec.transform(['Впечатления в итоге сдержанные и смешанные. Фильм меня не захватил. По большей части, два часа в кадре показывают действия и озвучивают мысли одного человека, который не вызвал у меня симпатии сразу, показался старше, чем ему прописано быть в сценарии. Не захотелось ему сопереживать. Сходу возникло предположение, что от него следует ожидать каких-то неадекватных поступков.'])
lr.predict(my_vec)

array([0])

## Использование разных классификаторов

### Классификация с помощью наивного Байесовского классификатора

In [59]:
from sklearn.naive_bayes import MultinomialNB

In [60]:
vec = CountVectorizer()
bow = vec.fit_transform(df.text)
train_data, test_data, train_labels, test_labels = train_test_split(bow, df.label) 

In [61]:
#Инициализируем Байесовский классификатор: 
nb = MultinomialNB()
nb.fit(train_data, train_labels)
predicted_labels = nb.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.70      0.22      0.33      1110
           0       0.35      0.07      0.12      1464
           1       0.81      0.98      0.89      9052

    accuracy                           0.79     11626
   macro avg       0.62      0.42      0.45     11626
weighted avg       0.74      0.79      0.74     11626



Модель, обученная с помощью наивного байесовского классификатора показало качество _хуже_, чем с логрегрессией. 

### Классификация с помощью деревьев решений

Идея метода: каждый признак - это критерий, это критерий, чтобы выбрать к какому классу относится объект. Мы можем построить дерево, где каждый узел - это разветвление по признакам. 
Корень - самый значимый признак, а дальше другие.

In [62]:
from sklearn.tree import DecisionTreeClassifier

In [63]:
dtc = DecisionTreeClassifier()
dtc.fit(train_data, train_labels)
predicted_labels = dtc.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.26      0.22      0.24      1110
           0       0.18      0.14      0.16      1464
           1       0.82      0.86      0.84      9052

    accuracy                           0.71     11626
   macro avg       0.42      0.41      0.41     11626
weighted avg       0.68      0.71      0.69     11626



Модель, обученная с помощью алгоритма деревьев решений показало качество _хуже_, чем с логрегрессией.

### Классификация с помощью метода k ближайших соседей

Идея метода: это предположение о том, что схожие (близкие в пространстве признаков) объекты гораздо чаще лежат в одном классе, чем в разных

In [64]:
from sklearn.neighbors import KNeighborsClassifier

In [65]:
?KNeighborsClassifier

Init signature:
KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None,
    **kwargs,
)
Docstring:     
Classifier implementing the k-nearest neighbors vote.

Read more in the :ref:`User Guide <classification>`.

Parameters
----------
n_neighbors : int, optional (default = 5)
    Number of neighbors to use by default for :meth:`kneighbors` queries.

weights : str or callable, optional (default = 'uniform')
    weight function used in prediction.  Possible values:

    - 'uniform' : uniform weights.  All points in each neighborhood
      are weighted equally.
    - 'distance' : weight points by the inverse of their distance.
      in this case, closer neighbors of a query point will have a
      greater influence than neighbors which are further away.
    - [callable] : a user-defined function which accepts an
      array of distances, and returns an array of the sam

In [66]:
knn = KNeighborsClassifier()
knn.fit(train_data, train_labels)
predicted_labels = knn.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.22      0.13      0.16      1110
           0       0.16      0.12      0.14      1464
           1       0.80      0.88      0.84      9052

    accuracy                           0.71     11626
   macro avg       0.40      0.37      0.38     11626
weighted avg       0.66      0.71      0.68     11626



Модель, обученная с помощью метода k ближайших соседей, показало качество _хуже_, чем с логрегрессией. 

### Попробуем другой линейный классификатор - SGDClassifier

In [67]:
sgdcl = SGDClassifier(random_state=1)
sgdcl.fit(train_data, train_labels)
predicted_labels = sgdcl.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.66      0.43      0.52      1110
           0       0.32      0.32      0.32      1464
           1       0.88      0.92      0.90      9052

    accuracy                           0.80     11626
   macro avg       0.62      0.56      0.58     11626
weighted avg       0.79      0.80      0.79     11626



Качество хуже, чем у логрегрессии. 

### Попробуем еще один линейный классификатор LinearSVC

In [69]:
from sklearn.svm import LinearSVC

In [70]:
l_SVC = LinearSVC(random_state=1)
l_SVC.fit(train_data, train_labels)
predicted_labels = l_SVC.predict(test_data)
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.61      0.53      0.57      1110
           0       0.35      0.26      0.30      1464
           1       0.88      0.93      0.91      9052

    accuracy                           0.81     11626
   macro avg       0.61      0.58      0.59     11626
weighted avg       0.79      0.81      0.80     11626



/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Качество хуже, чем у логрегрессии. 